## Imports

In [1]:
import pandas as pd
import numpy as np

# Geo processing
import shapely
import geopandas as gpd

## Read 2014 Uber Data into GeoPandaDataframe

Read Uber data as pandas dataframe

In [2]:
df = pd.read_csv('./uber-raw-data-apr14.csv', parse_dates = ['Date/Time'])
df.head()

,Date/Time,Lat,Lon,Base
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512


Convert to geopandas dataframe, with (Lon, Lat) points

In [3]:
apr14_gdf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.Lon, df.Lat))
apr14_gdf.head()

,Date/Time,Lat,Lon,Base,geometry
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512,POINT (-73.95490 40.76900)
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,POINT (-74.03450 40.72670)
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512,POINT (-73.98730 40.73160)
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512,POINT (-73.97760 40.75880)
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512,POINT (-73.97220 40.75940)


In [4]:
apr14_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 564516 entries, 0 to 564515
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   Date/Time  564516 non-null  datetime64[ns]
 1   Lat        564516 non-null  float64       
 2   Lon        564516 non-null  float64       
 3   Base       564516 non-null  object        
 4   geometry   564516 non-null  geometry      
dtypes: datetime64[ns](1), float64(2), geometry(1), object(1)
memory usage: 21.5+ MB


Create a sample of size 1000 for testing purposes

In [5]:
apr14_gdf_test = apr14_gdf[:1000].copy()
apr14_gdf_test.shape

(1000, 5)

## Get taxi zones from shp file

Obtain taxi zones from the shp file (source: https://data.cityofnewyork.us/Transportation/NYC-Taxi-Zones/d3c5-ddgc)

In [6]:
import geopandas as gpd

zones_gdf = gpd.read_file("./NYC Taxi Zones/geo_export_e4414726-1909-4bd6-9087-38ad3b47f148.shp")
zones_gdf.head()

,borough,location_i,objectid,shape_area,shape_leng,zone,geometry
0,EWR,1.0,1.0,0.000782,0.116357,Newark Airport,"POLYGON ((-74.18445 40.69500, -74.18449 40.695..."
1,Queens,2.0,2.0,0.004866,0.433470,Jamaica Bay,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,Bronx,3.0,3.0,0.000314,0.084341,Allerton/Pelham Gardens,"POLYGON ((-73.84793 40.87134, -73.84725 40.870..."
3,Manhattan,4.0,4.0,0.000112,0.043567,Alphabet City,"POLYGON ((-73.97177 40.72582, -73.97179 40.725..."
4,Staten Island,5.0,5.0,0.000498,0.092146,Arden Heights,"POLYGON ((-74.17422 40.56257, -74.17349 40.562..."


Note about columns: 
* objectid is the unique identifier for each row
* location_i is the locationID we want to obtain. The same location_i values can be associated with different geometries in the table

In [8]:
# Exploration: What is the objectid column is it the same as the location_id or the index? 
zones_gdf[zones_gdf.objectid != zones_gdf.location_i]

,borough,location_i,objectid,shape_area,shape_leng,zone,geometry
56,Queens,56.0,57.0,0.000018,0.019271,Corona,"POLYGON ((-73.85131 40.74984, -73.85443 40.748..."
104,Manhattan,103.0,104.0,0.000012,0.021221,Governor's Island/Ellis Island/Liberty Island,"POLYGON ((-74.03995 40.70089, -74.03945 40.700..."
105,Manhattan,103.0,105.0,0.000369,0.077425,Governor's Island/Ellis Island/Liberty Island,"POLYGON ((-74.01675 40.69334, -74.01540 40.693..."


In [9]:
# Exploration: duplicationd location_i values
zones_gdf[zones_gdf.location_i == 103.0]

,borough,location_i,objectid,shape_area,shape_leng,zone,geometry
103,Manhattan,103.0,103.0,0.000006,0.014306,Governor's Island/Ellis Island/Liberty Island,"POLYGON ((-74.04389 40.69018, -74.04351 40.689..."
104,Manhattan,103.0,104.0,0.000012,0.021221,Governor's Island/Ellis Island/Liberty Island,"POLYGON ((-74.03995 40.70089, -74.03945 40.700..."
105,Manhattan,103.0,105.0,0.000369,0.077425,Governor's Island/Ellis Island/Liberty Island,"POLYGON ((-74.01675 40.69334, -74.01540 40.693..."


In [11]:
# Exploration: which location_i values are duplicated
zones_gdf.groupby('location_i').count().sort_values(by = ['borough'], ascending = False).head()

,borough,objectid,shape_area,shape_leng,zone,geometry
location_i,,,,,,
103.0,3,3,3,3,3,3
56.0,2,2,2,2,2,2
1.0,1,1,1,1,1,1
176.0,1,1,1,1,1,1
181.0,1,1,1,1,1,1


Set objectid as the index

In [12]:
# Set the objectid as the index
zones_gdf = zones_gdf.set_index('objectid')
zones_gdf.head()

,borough,location_i,shape_area,shape_leng,zone,geometry
objectid,,,,,,
1.0,EWR,1.0,0.000782,0.116357,Newark Airport,"POLYGON ((-74.18445 40.69500, -74.18449 40.695..."
2.0,Queens,2.0,0.004866,0.433470,Jamaica Bay,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
3.0,Bronx,3.0,0.000314,0.084341,Allerton/Pelham Gardens,"POLYGON ((-73.84793 40.87134, -73.84725 40.870..."
4.0,Manhattan,4.0,0.000112,0.043567,Alphabet City,"POLYGON ((-73.97177 40.72582, -73.97179 40.725..."
5.0,Staten Island,5.0,0.000498,0.092146,Arden Heights,"POLYGON ((-74.17422 40.56257, -74.17349 40.562..."


## Conversion from lon/lat Points to Taxi Zones

Helper function to convert coordinates to taxi zones

In [13]:
def coord_to_zone(point): 
    ''' Given a point, identifies the locationID that it's contained in'''
    
    # Vector indicating which of the zones contains the given point
    contained_in_zone = zones_gdf['geometry'].contains(point)
    
    # Point is not contained in any of the zones. LocationID = 265 
    if max(contained_in_zone) == 0: 
        locationID = 265
    
    # Finds the zone the point is contained in (where vector = True)
    # Assumes a point does not land in multiple zones
    else: 
        # Relies on truthy/ falsey (i.e. 1/0) conversion
        location_index = np.argmax(contained_in_zone) 
        locationID = zones_gdf.iloc[location_index]['location_i']
        
    return int(locationID)

Test coord_to_zone

In [18]:
# Test case
from shapely.geometry import Polygon, LineString, Point

# point within NYC
test_point1 = Point(-73.95490, 40.76900) 
coord_to_zone_return = coord_to_zone(test_point1)
print(zones_gdf[zones_gdf['location_i'] == coord_to_zone_return]['geometry']\
                                        .contains(test_point1))

print()

# point outside of NYC
test_point2 = Point(-81.5158, 27.6648)
coord_to_zone_return = coord_to_zone(test_point2)
print(coord_to_zone_return)

objectid
140.0    True
dtype: bool

265


## Transform coordinates to zones (uber 2014 data)

Note: This is currently only being applied to 1000 records. It would take a VERY long time to apply it to all the records. Thus, we need to recreate this pipeline using pyspark. 

In [19]:
apr14_gdf_test['locationID'] = apr14_gdf_test['geometry'].apply(lambda x: coord_to_zone(x))
apr14_gdf_test.head()

,Date/Time,Lat,Lon,Base,geometry,locationID
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512,POINT (-73.95490 40.76900),140
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,POINT (-74.03450 40.72670),265
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512,POINT (-73.98730 40.73160),79
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512,POINT (-73.97760 40.75880),161
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512,POINT (-73.97220 40.75940),162
